# Rate Limiting in FastAPI

## Installation

The slowapi library is a popular option for adding rate limiting to FastAPI. It integrates seamlessly with FastAPI using middleware and decorators.

In [ ]:
pip install slowapi

## Basic Example

In [ ]:
from fastapi import FastAPI, Request
from slowapi import Limiter
from slowapi.util import get_remote_address
from slowapi.middleware import SlowAPIMiddleware
from slowapi.errors import RateLimitExceeded
from starlette.responses import JSONResponse

# Initialize FastAPI app and Limiter
app = FastAPI()
limiter = Limiter(key_func=get_remote_address)  # Rate limit based on client IP
app.state.limiter = limiter
app.add_middleware(SlowAPIMiddleware)  # Add the SlowAPI middleware

# Global error handler for rate limit exceeded
@app.exception_handler(RateLimitExceeded)
async def rate_limit_exceeded_handler(request: Request, exc: RateLimitExceeded):
    return JSONResponse(
        status_code=429, 
        content={"detail": "Rate limit exceeded. Try again later."}
    )

# Example route with rate limiting
@app.get("/items/")
@limiter.limit("5/minute")  # 5 requests per minute
async def read_items(request: Request):
    return {"message": "This route has rate limiting"}


#### Key Points:
- **key_func**: Defines how rate limiting is applied (e.g., by IP, user ID, API key, etc.).
- **Decorator**: Use @limiter.limit to set limits like "10/second" or "5/minute".
- **Global Middleware**: Required for the slowapi library to work.